In [39]:
import openai
import clip
import torch
from PIL import Image
import os
from google.cloud import vision



In [30]:
# Load the model onto the CPU
model, preprocess = clip.load("ViT-B/32", device='cpu')

# Prepare the image and text
# Make sure to use the correct path to your image file
image_path = "Example Data-20240208T214429Z-001/Example Data/exported/IMG_3614 Large.jpeg"  # Replace with your actual image path
image = preprocess(Image.open(image_path)).unsqueeze(0).to('cpu')
text = clip.tokenize(["tree "]).to('cpu')

# Calculate the features
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

# Compute the cosine similarity and convert to probabilities with softmax
similarity = (image_features @ text_features.T)

# Print the similarity score
print("Similarity score:", similarity.numpy().flatten())

Similarity score: [21.849493]


In [19]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Download the dataset
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

# Prepare the inputs
image, class_id = cifar100[3637]
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")

Files already downloaded and verified

Top predictions:

           snake: 65.31%
          turtle: 12.29%
    sweet_pepper: 3.83%
          lizard: 1.88%
       crocodile: 1.75%


In [40]:
path = 'Example Data-20240208T214429Z-001/Example Data/exported/IMG_3614 Large.jpeg'
# Instantiate a client
client = vision.ImageAnnotatorClient()

# Load an image file
with open( path , 'rb') as image_file:
    content = image_file.read()
image = vision.Image(content=content)

# Perform safe search detection on the image file
response = client.safe_search_detection(image=image)
safe = response.safe_search_annotation

# Output detected properties
print('Adult: {}'.format(safe.adult))
print('Spoof: {}'.format(safe.spoof))
print('Medical: {}'.format(safe.medical))
print('Violence: {}'.format(safe.violence))
print('Racy: {}'.format(safe.racy))

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [41]:
!pip install requests


In [42]:
import requests
import json

In [46]:
# The URL where the DeepDetect server is running
DEEPDETECT_URL = 'http://localhost:8080/predict'

# The image you want to classify
IMAGE_PATH = 'Example Data-20240208T214429Z-001/Example Data/exported/(16) 18379894042056715.jpeg'



# The data for the POST request
data = {
    'service': 'nsfw',  # the name of the deployed NSFW model service
    'parameters': {
        'output': {
            'best': 1  # you can ask for more classes
        }
    },
    'data': [IMAGE_PATH]
}

# The headers for the POST request
headers = {
    'Content-Type': 'application/json'
}

# Perform the POST request to DeepDetect server
response = requests.post(DEEPDETECT_URL, headers=headers, data=json.dumps(data))

# Check the response status code
if response.status_code == 200:
    # Parse the JSON response
    response_json = response.json()
    predictions = response_json.get('predictions', [])

    if predictions:
        # Get the prediction result
        classes = predictions[0].get('classes')
        if classes:
            # Get the top class prediction
            top_class = classes[0]
            classname = top_class.get('cat')  # The category/class
            prob = top_class.get('prob')  # The probability/confidence
            print(f'Class: {classname}, Probability: {prob}')
        else:
            print('No classification results')
    else:
        print('No predictions in the response')
else:
    print('Request failed with status code', response.status_code)

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000019499526F60>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))